## Libraries

In [1]:
import pandas as pd
import numpy as np

## Load the data

In [2]:
train = pd.read_csv('../input/train.csv')
test = pd.read_csv('../input/test.csv')

In [3]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


## Data Processing

In [6]:
from nltk.corpus import stopwords

In [7]:
def word_match_share(row):
    stops_words = set(stopwords.words("english"))
    
    q1words = {}
    q2words = {}
    
    for word in str(row['question1']).lower().split():
        if word not in stops_words:
            q1words[word] = 1
            
    for word in str(row['question2']).lower().split():
        if word not in stops_words:
            q2words[word] = 1
            
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_words_in_q1 = [w for w in q1words.keys() if w in q2words]
    shared_words_in_q2 = [w for w in q2words.keys() if w in q1words]
    R = (len(shared_words_in_q1) + len(shared_words_in_q2))/(len(q1words) + len(q2words))
    return R

In [8]:
from multiprocessing import Pool

def transform_data(data):
    return data.apply(word_match_share, axis=1, raw=True)

# This takes about 2-4 minutes on i7 proc (desktop 2.7 GHz)
# TODO chunk more and increase process number, make a function

total = [train, test[:500000], test[500000:1000000], test[1000000:1500000], test[1500000:2000000], test[2000000:]]

pool = Pool(processes=8)
train['word_share'], test_c1, test_c2, test_c3, test_c4, test_c5 = pool.map(transform_data, total)  
test['word_share'] = np.hstack((test_c1, test_c2, test_c3, test_c4, test_c5))

# Old version without pool
#train['word_share'] = train.apply(word_match_share, axis=1, raw=True)
#test['word_share'] = test.apply(word_match_share, axis=1, raw=True)

In [9]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_share
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000


In [10]:
train[train['is_duplicate']==1][:20][['question1', 'question2']]

,question1,question2
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan..."
7,How can I be a good geologist?,What should I do to be a great geologist?
11,How do I read and find my YouTube comments?,How can I see all my Youtube comments?
12,What can make Physics easy to learn?,How can you make physics easy to learn?
13,What was your first sexual experience like?,What was your first sexual experience?
15,What would a Trump presidency mean for current...,How will a Trump presidency affect the student...
16,What does manipulation mean?,What does manipulation means?
18,Why are so many Quora users posting questions ...,Why do people ask Quora questions which can be...
20,Why do rockets look white?,Why are rockets and boosters painted white?
29,How should I prepare for CA final law?,How one should know that he/she completely pre...


In [11]:
test.head()

,test_id,question1,question2,word_share
0,0,How does the Surface Pro himself 4 compare wit...,Why did Microsoft choose core m3 and not core ...,0.266667
1,1,Should I have a hair transplant at age 24? How...,How much cost does hair transplant require?,0.500000
2,2,What but is the best way to send money from Ch...,What you send money to China?,0.444444
3,3,Which food not emulsifiers?,What foods fibre?,0.000000
4,4,"How ""aberystwyth"" start reading?",How their can I start reading?,0.800000


In [12]:
def start_with_same_first_word(row):
    if not isinstance(row['question1'], str) or not isinstance(row['question2'], str):
        return 0
    
    first_word_q1 = row['question1'].split()[0].lower()
    first_word_q2 = row['question2'].split()[0].lower()
    
    return 1 if first_word_q1 == first_word_q2 else 0

train['start_with_same_world'] = train.apply(start_with_same_first_word, axis=1, raw=True)
test['start_with_same_world'] = test.apply(start_with_same_first_word, axis=1, raw=True)

In [13]:
# This takes about 3-4 minute on i5 proc (desktop 3.5 GHz)

train['q1_char_num'] = train.apply(lambda x: len(x['question1']), axis=1, raw=True)
train['q1_word_num'] = train.apply(lambda x: len(x['question1'].split()), axis=1, raw=True)

train['q2_char_num'] = train.apply(
    lambda x: len(x['question2']) if isinstance(x['question2'], str) else 0,axis=1, raw=True
)
train['q2_word_num'] = train.apply(
    lambda x: len(x['question2'].split()) if isinstance(x['question2'], str) else 0, axis=1, raw=True
)

test['q1_char_num'] = test.apply(
    lambda x: len(x['question1']) if isinstance(x['question1'], str) else 0,axis=1, raw=True
)
test['q1_word_num'] = test.apply(
    lambda x: len(x['question1'].split()) if isinstance(x['question1'], str) else 0, axis=1, raw=True
)

test['q2_char_num'] = test.apply(
    lambda x: len(x['question2']) if isinstance(x['question2'], str) else 0,axis=1, raw=True
)
test['q2_word_num'] = test.apply(
    lambda x: len(x['question2'].split()) if isinstance(x['question2'], str) else 0, axis=1, raw=True
)

In [14]:
from collections import Counter

train_qs = pd.Series(train['question1'].tolist() + train['question2'].tolist()).astype(str)
# If a word appears only once, we ignore it completely (likely a typo)
# Epsilon defines a smoothing constant, which makes the effect of extremely rare words smaller
def get_weight(count, eps=10000, min_count=2):
    if count < min_count:
        return 0
    else:
        return 1 / (count + eps)

eps = 5000 
words = (" ".join(train_qs)).lower().split()
counts = Counter(words)
weights = {word: get_weight(count) for word, count in counts.items()}

In [15]:
import gc

del train_qs
gc.collect()

332

In [16]:
import math

def tfidf_word_match_share(row):
    stops_words = set(stopwords.words("english"))
    
    q1words = {}
    q2words = {}
    
    for word in str(row['question1']).lower().split():
        if word not in stops_words:
            q1words[word] = 1
            
    for word in str(row['question2']).lower().split():
        if word not in stops_words:
            q2words[word] = 1
            
    if len(q1words) == 0 or len(q2words) == 0:
        # The computer-generated chaff includes a few questions that are nothing but stopwords
        return 0
    
    shared_weights = [weights.get(w, 0) for w in q1words.keys() if w in q2words] + [weights.get(w, 0) for w in q2words.keys() if w in q1words]
    total_weights = [weights.get(w, 0) for w in q1words] + [weights.get(w, 0) for w in q2words]
    
    R = np.sum(shared_weights) / np.sum(total_weights)
    
    return R if not math.isnan(R) else 0

train['rfidf_share'] = train.apply(tfidf_word_match_share, axis=1, raw=True)
test['rfidf_share'] = test.apply(tfidf_word_match_share, axis=1, raw=True)

/home/weenkus/.conda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in double_scalars
/home/weenkus/.conda/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:24: RuntimeWarning: invalid value encountered in long_scalars


In [17]:
train.head()

,id,qid1,qid2,question1,question2,is_duplicate,word_share,start_with_same_world,q1_char_num,q1_word_num,q2_char_num,q2_word_num,rfidf_share
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,0.727273,1,66,14,57,12,0.772164
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,0.307692,1,51,8,88,13,0.361758
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,0.363636,1,73,14,59,10,0.355191
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,0.000000,0,50,11,65,9,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,0.000000,1,76,13,39,7,0.000000


In [18]:
train['char_difference'] = abs(train['q1_char_num'] - train['q2_char_num'])
train['word_difference'] = abs(train['q1_word_num'] - train['q2_word_num'])
test['char_difference'] = abs(test['q1_char_num'] - test['q2_char_num'])
test['word_difference'] = abs(test['q1_word_num'] - test['q2_word_num'])

## Feature picking

In [45]:
try:
    from sklearn.model_selection import train_test_split
except ImportError:
    from sklearn.cross_validation import train_test_split

features = ['word_share', 'q1_char_num', 'q1_word_num', 'q2_char_num', 'q2_word_num',
            'start_with_same_world', 'rfidf_share']

#features = ['word_share', 'q1_char_num', 'q1_word_num', 'q2_char_num', 'q2_word_num',
#            'start_with_same_world', 'rfidf_share', 'char_difference', 'word_difference']

target = 'is_duplicate'

X = train[features]
y = train[target]

## Oversampling

In [46]:
pos_train = X[y == 1]
neg_train = X[y == 0]

# Now we oversample the negative class
# There is likely a much more elegant way to do this...
p = 0.165
scale = ((len(pos_train) / (len(pos_train) + len(neg_train))) / p) - 1
while scale > 1:
    neg_train = pd.concat([neg_train, neg_train])
    scale -=1
neg_train = pd.concat([neg_train, neg_train[:int(scale * len(neg_train))]])
print(len(pos_train) / (len(pos_train) + len(neg_train)))

X = pd.concat([pos_train, neg_train])
y = (np.zeros(len(pos_train)) + 1).tolist() + np.zeros(len(neg_train)).tolist()

del pos_train, neg_train

0.19124366100096607


## Cross validation

In [47]:
X_train, X_vald, y_train, y_vald = train_test_split(X, y, test_size=0.2, random_state=42)
X_test = test[features]

In [48]:
X_test.head()

,word_share,q1_char_num,q1_word_num,q2_char_num,q2_word_num,start_with_same_world,rfidf_share
0,0.266667,57,11,68,14,0,0.274019
1,0.500000,66,14,43,7,0,0.480962
2,0.444444,60,14,29,6,1,0.468893
3,0.000000,27,4,17,3,0,0.000000
4,0.800000,32,4,30,6,1,1.000000


In [49]:
X.head()

,word_share,q1_char_num,q1_word_num,q2_char_num,q2_word_num,start_with_same_world,rfidf_share
5,0.470588,86,16,90,16,0,0.510771
7,0.500000,30,7,41,9,0,0.645836
11,0.571429,43,9,38,8,1,0.654071
12,1.000000,36,7,39,8,0,1.000000
13,0.571429,43,7,38,6,1,0.536457


In [35]:
#del train, test
#gc.collect()

## Transofmrations

In [65]:
from sklearn.preprocessing import StandardScaler

In [66]:
# Model works fine without scaling

scaler = StandardScaler().fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_vald_scaled = scaler.transform(X_vald)

## Model

In [70]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score, log_loss

from xgboost import XGBClassifier
#import lightgbm

from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

In [56]:
#model = RandomForestClassifier(n_estimators=32, n_jobs=8)
#model = ExtraTreesClassifier(n_estimators=32, n_jobs=8)
#model = AdaBoostClassifier()
#model = GradientBoostingClassifier(n_estimators=420) 0.46
#model = KNeighborsClassifier(n_neighbors=25)
#model = MultinomialNB() 0.57
#model = SVC()

model = XGBClassifier(n_estimators=500, learning_rate=0.48, max_depth=15)
model.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_vald, y_vald)],
          early_stopping_rounds=50, verbose=True, eval_metric='logloss')


#model = VotingClassifier(estimators=[('xgb', xgb), ('knn', knn), ('rf', rf)],
#                        voting='soft', weights=[4.5, 1.1, 1.2])

#model.fit(X_train, y_train)

[0]	validation_0-logloss:0.489049	validation_1-logloss:0.494666
Multiple eval metrics have been passed: 'validation_1-logloss' will be used for early stopping.

Will train until validation_1-logloss hasn't improved in 50 rounds.
[1]	validation_0-logloss:0.408084	validation_1-logloss:0.418434
[2]	validation_0-logloss:0.364922	validation_1-logloss:0.380379
[3]	validation_0-logloss:0.340922	validation_1-logloss:0.359863
[4]	validation_0-logloss:0.325383	validation_1-logloss:0.347637
[5]	validation_0-logloss:0.315288	validation_1-logloss:0.340392
[6]	validation_0-logloss:0.309037	validation_1-logloss:0.336339
[7]	validation_0-logloss:0.300966	validation_1-logloss:0.332918
[8]	validation_0-logloss:0.296302	validation_1-logloss:0.330703
[9]	validation_0-logloss:0.293789	validation_1-logloss:0.329613
[10]	validation_0-logloss:0.29074	validation_1-logloss:0.328562
[11]	validation_0-logloss:0.289883	validation_1-logloss:0.328318
[12]	validation_0-logloss:0.286986	validation_1-logloss:0.327476
[

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.48, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [58]:
val_predictions = model.predict(X_vald)
val_prob_predictions = model.predict_proba(X_vald)

for metric_name, metric_func in zip(
    ['F1-score', 'Acc', 'Precision', 'Recall', 'LogLoss'],
    [f1_score, accuracy_score, precision_score, recall_score, log_loss]
):
    
    val_predictions = val_predictions if metric_name not in ['LogLoss'] else val_prob_predictions
    metric_score = metric_func(y_vald, val_predictions)
    print('{0}: {1}'.format(metric_name, metric_score))

F1-score: 0.6606727702911224
Acc: 0.8943420159130803
Precision: 0.8504237288135593
Recall: 0.5401513877207738
LogLoss: 0.2630174460680658


In [59]:
model.fit(X, y)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.48, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=500, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [45]:
#del X, y, X_train, y_train, X_vald, y_vald
#gc.collect()

369

In [35]:
X_test.head()

,word_share,q1_char_num,q1_word_num,q2_char_num,q2_word_num,start_with_same_world,rfidf_share
0,0.266667,57,11,68,14,0,0.274019
1,0.500000,66,14,43,7,0,0.480962
2,0.444444,60,14,29,6,1,0.468893
3,0.000000,27,4,17,3,0,0.000000
4,0.800000,32,4,30,6,1,1.000000


In [60]:
predictions = model.predict_proba(X_test)

In [84]:
predictions[:,1]

array([ 0.05369232,  0.64482141,  0.52798671, ...,  0.00065555,
        0.10855845,  0.46532747], dtype=float32)

## Generate submission

In [61]:
np.savetxt(
    'submission.csv', np.c_[range(len(predictions)), predictions[:,1]],
    delimiter=',', header='test_id,is_duplicate', comments='', fmt='%d,%f'
)

In [ ]:
with open('submission.csv', 'w', buffering=1) as submission_file:
    submission_file.write('test_id,is_duplicate')
    
    for test_id, test_row in enumerate(X_test.iterrows()):
        row_prediction = model.predict_proba(X_test[test_id:])
        submission_file.write('%d,%f' % test_id, row_prediction[:,1])